# 1. RAG Workshop: Build a simple Q&A RAG

In this task, you’ll create a question-and-answer system using MistralAI’s language models. Learn to:

- Use MistralAI LLMs for generating answers.
- Enrich responses by adding external knowledge to prompts.
- Optimize prompts through experimentation for better accuracy.

By the end, you built a simple Q&A systems


## Requirements

In [3]:
print("hallo")

hallo


In [4]:
1+1==2

True

In [23]:
import os
import PyPDF2

from mistralai import Mistral
from dotenv import load_dotenv, find_dotenv

## TODO: Set Up Access to MistralAI

We'll be using Mistral, a popular OpenAI LLM, to build our Q&A system. First, let's set up access by creating an account and generating an API key. Follow the steps below to get started.

1. **Create an account** on [MistralAI](https://mistral.ai/) if you don’t have one.
2. **Log in to the MistralAI console**: Go to [https://console.mistral.ai/api-keys/](https://console.mistral.ai/api-keys/).
3. **Generate an API key**: Click to create a new API Key.
4. **Save your API key**: Copy the API key and create a `.env` file in the root of the project repository.
5. **Add the API key to your `.env` file**. Your `.env` file should look like this: `MISTRAL_API_KEY=YOUR_API_TOKEN`

In [11]:
load_dotenv(find_dotenv())

def todo_setup_access_completed():
    token = os.getenv('MISTRAL_API_KEY')
    if token is None:
        raise Exception(".env is not in the root folder or `MISTRAL_API_KEY` is not set.")

todo_setup_access_completed()


## TODO: Call the Mistral API

In this section, we will learn how to call open-source models using the `mistral-small-latest` model, which is smaller and faster. We will utilize [Mistral’s open-source Python client](https://github.com/mistralai/client-python), to complete the coding sections. For more information, refer to the [documentation on the Python client](https://docs.mistral.ai/getting-started/clients/).

In [15]:
mistral_api_key = os.getenv('MISTRAL_API_KEY')
mistral_client = Mistral(api_key=mistral_api_key)
mistral_model = "mistral-small-latest"

In [16]:
def call_mistral_api(client: Mistral, model: str, message: str) -> str:
    # TODO: Use the client to call MistralAPI and respond the message as string
    
    response = client.chat.complete(
        model = model,
        messages = [
            {
                "role": "user",
                "content": message,
            }
        ]
    )
    return response.choices[0].message.content

In [21]:
# Testing if it is successful
print(call_mistral_api(client=mistral_client, model=mistral_model, message="Who is the current US president?"))

As of my last update in October 2023, the current President of the United States is Joe Biden. He took office on January 20, 2021. However, for the most current information, please refer to a reliable and up-to-date source.


## TODO: Simple Q&A RAG

Large language models (LLMs) can sometimes hallucinate, presenting false information due to outdated training data. Retrieval-Augmented Generation (RAG) allows us to incorporate external information to mitigate these challenges. In this task, we will create a simple Q&A RAG that utilizes knowledge from a PDF to enrich its answers.

In [29]:
def extract_text_from_pdf(pdf_path: str) -> str:
    # TODO: Use PyPDF2 to load a PDF as text https://pypdf2.readthedocs.io/en/3.x/user/extract-text.html
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text() or ""  # Handle cases where text extraction might fail
            page_text = page_text.replace("\t", " ")
            text += page_text
    return text

text = extract_text_from_pdf("../data/food_lab_green_chapter-small.pdf")


Now that we have the text, we can begin enriching the prompt to make our LLM even smarter!

In [31]:
def create_rag_prompt(message, context):
    return f"""Answer the question only using the provided content.

        Context: {context}

        User Question: {message}

        Respond in a natural way. If the information cannot be found in the context, respond with "It is out of my pay grade" and be more rude.
        """  

Below you can compare how our LLM differs the answers by the information that you provided

In [33]:
def compare_llm_answers(message):
    generic_response = call_mistral_api(client=mistral_client, model=mistral_model, message=message)
    
    rag_prompt = create_rag_prompt(message=message, context=text)
    rag_response = call_mistral_api(client=mistral_client, model=mistral_model, message=rag_prompt)

    print(f"GENERIC RESPONSE:\n {generic_response}")
    print("-" * 10)
    print(f"RAG RESPONSE:\n {rag_response}")

compare_llm_answers("How do you design a salad recipe")

GENERIC RESPONSE:
 Designing a salad recipe involves considering various elements like ingredients, flavors, textures, and presentation. Here's a step-by-step guide to help you create your own salad recipe:

1. **Choose a Base:**
   - Leafy greens: Mix greens, spinach, arugula, romaine, kale, etc.
   - Grains: Quinoa, couscous, farro, or rice.
   - Other bases: Pasta, beans, or lentils.

2. **Add Proteins (optional):**
   - Animal-based: Chicken, turkey, beef, pork, fish, shrimp, eggs, or cheese.
   - Plant-based: Tofu, tempeh, chickpeas, lentils, or nuts/seeds.

3. **Include Veggies:**
   - Raw: Carrots, cucumbers, bell peppers, radishes, etc.
   - Cooked: Roasted vegetables like sweet potatoes, beets, or Brussels sprouts.
   - Pickled or fermented: Sauerkraut, pickles, or kimchi.

4. **Add Fruits (optional):**
   - Fresh: Apples, berries, grapes, oranges, etc.
   - Dried: Cranberries, raisins, or apricots.

5. **Select Cheeses (optional):**
   - Crumbly: Feta, goat cheese, or blue ch

That's it! RAGs enrich the prompt with additional information about the topic to generate responses. The external information can come from various sources, not just PDFs, such as Google search results, social media posts, and more. With that, we’ve built a simple Q&A RAG. In the next chapter, we will scale it up to include even more context.